In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


# Dynamic Graphs

In [3]:
node_list = np.array(list(g.nodes.keys()))
# the i-th node in node_list corresponds to the i-th row of Adjacency matrix

# from node to index in node_list
node_list_index_dict = {}
for node in g.nodes:
    node_list_index_dict[node] = np.where(node_list == node)[0][0]

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]

In [5]:

top_prop = 0.5

adjacency_list = []

A = nx.adjacency_matrix(g).todense()


for rep in tqdm(range(len(ramps))):
    np.random.seed(0)
    
    degrees = A.sum(axis=1)
    deg_bar = np.quantile(degrees, 1-top_prop)
    evolve_nodes_index = np.where(degrees >= deg_bar)[0]
    deg_prob = degrees / degrees.sum()
    
    for i in evolve_nodes_index:    
        connect_node_indices = np.random.choice(range(num_nodes), size = 5, p=deg_prob)
        
        for connect_node_idx in connect_node_indices:
            # i_node_key = node_list[i]
            # conn_node_key = node_list[connect_node_idx]
            i_node_key = i 
            conn_node_key = connect_node_idx
            
            if i_node_key == conn_node_key:
                continue
            if A[i_node_key][conn_node_key] == 0:
                A[i_node_key][conn_node_key] = 1
                A[conn_node_key][i_node_key] = 1
            
    adjacency_list.append(A.copy())

100%|██████████| 5/5 [00:20<00:00,  4.14s/it]


In [6]:
g_list = [nx.from_numpy_array(adj_mat) for adj_mat in adjacency_list]

In [7]:
for adj_mat in adjacency_list:
    print(adj_mat.sum()//2)

596414
621807
648583
671663
698233


# Model Training

In [8]:
dgl_G_list = [dgl.from_networkx(g_) for g_ in g_list]

In [9]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [10]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(1)
        g_feat_list = []
        y_list = []        

        for step_idx in range(num_step, len(ramps)):
            g = g_list[step_idx]
            dgl_G = dgl_G_list[step_idx]
            p = ramps[step_idx]
            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]            
            tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])  
                        
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.degree[n]] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(ramps) - num_step):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {ramps[num_step]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/Dynamic_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 0.3894
Epoch 0, Treat Prop: 0.02, Loss: 0.2510
Epoch 0, Treat Prop: 0.02, Loss: 0.3658
Epoch 0, Treat Prop: 0.02, Loss: 0.4671
Epoch 0, Treat Prop: 0.02, Loss: 1.2240
Epoch 50, Treat Prop: 0.02, Loss: 0.0565
Epoch 50, Treat Prop: 0.02, Loss: 0.1041
Epoch 50, Treat Prop: 0.02, Loss: 0.0676
Epoch 50, Treat Prop: 0.02, Loss: 0.0580
Epoch 50, Treat Prop: 0.02, Loss: 0.3293
Epoch 100, Treat Prop: 0.02, Loss: 0.0134
Epoch 100, Treat Prop: 0.02, Loss: 0.0225
Epoch 100, Treat Prop: 0.02, Loss: 0.0239
Epoch 100, Treat Prop: 0.02, Loss: 0.0282
Epoch 100, Treat Prop: 0.02, Loss: 0.0622
Epoch 150, Treat Prop: 0.02, Loss: 0.0364
Epoch 150, Treat Prop: 0.02, Loss: 0.0191
Epoch 150, Treat Prop: 0.02, Loss: 0.0253
Epoch 150, Treat Prop: 0.02, Loss: 0.0417
Epoch 150, Treat Prop: 0.02, Loss: 0.0373
Epoch 200, Treat Prop: 0.02, Loss: 0.0160
Epoch 200, Treat Prop: 0.02, Loss: 0.0299
Epoch 200, Treat Prop: 0.02, Loss: 0.0257
Epoch 200, Treat Prop: 0.02, Loss: 0.0232
Epoch 2

  0%|          | 1/1000 [00:37<10:19:33, 37.21s/it]

...

0.029806507751345634
Seed: 387
Epoch 0, Treat Prop: 0.02, Loss: 0.3757
Epoch 0, Treat Prop: 0.02, Loss: 0.2733
Epoch 0, Treat Prop: 0.02, Loss: 0.3782
Epoch 0, Treat Prop: 0.02, Loss: 0.4463
Epoch 0, Treat Prop: 0.02, Loss: 0.8706
Epoch 50, Treat Prop: 0.02, Loss: 0.0502
Epoch 50, Treat Prop: 0.02, Loss: 0.0808
Epoch 50, Treat Prop: 0.02, Loss: 0.0486
Epoch 50, Treat Prop: 0.02, Loss: 0.0620
Epoch 50, Treat Prop: 0.02, Loss: 0.2201
Epoch 100, Treat Prop: 0.02, Loss: 0.0182
Epoch 100, Treat Prop: 0.02, Loss: 0.0428
Epoch 100, Treat Prop: 0.02, Loss: 0.0239
Epoch 100, Treat Prop: 0.02, Loss: 0.0347
Epoch 100, Treat Prop: 0.02, Loss: 0.0695
Epoch 150, Treat Prop: 0.02, Loss: 0.0233
Epoch 150, Treat Prop: 0.02, Loss: 0.0163
Epoch 150, Treat Prop: 0.02, Loss: 0.0226
Epoch 150, Treat Prop: 0.02, Loss: 0.0325
Epoch 150, Treat Prop: 0.02, Loss: 0.0327
Epoch 200, Treat Prop: 0.02, Loss: 0.0136
Epoch 200, Treat Prop: 0.02, Loss: 0.0200
Epoch 200, Treat Prop: 0.02, Loss: 0.0156
Epoch 200, Treat P

 39%|███▉      | 388/1000 [3:46:11<5:54:52, 34.79s/it]

0.024035224691033363
Seed: 388
Epoch 0, Treat Prop: 0.02, Loss: 0.3821
Epoch 0, Treat Prop: 0.02, Loss: 0.2598
Epoch 0, Treat Prop: 0.02, Loss: 0.3707
Epoch 0, Treat Prop: 0.02, Loss: 0.5480
Epoch 0, Treat Prop: 0.02, Loss: 1.3121
Epoch 50, Treat Prop: 0.02, Loss: 0.0654
Epoch 50, Treat Prop: 0.02, Loss: 0.1099
Epoch 50, Treat Prop: 0.02, Loss: 0.0706
Epoch 50, Treat Prop: 0.02, Loss: 0.0706
Epoch 50, Treat Prop: 0.02, Loss: 0.3322
Epoch 100, Treat Prop: 0.02, Loss: 0.0148
Epoch 100, Treat Prop: 0.02, Loss: 0.0276
Epoch 100, Treat Prop: 0.02, Loss: 0.0243
Epoch 100, Treat Prop: 0.02, Loss: 0.0339
Epoch 100, Treat Prop: 0.02, Loss: 0.0666
Epoch 150, Treat Prop: 0.02, Loss: 0.0244
Epoch 150, Treat Prop: 0.02, Loss: 0.0206
Epoch 150, Treat Prop: 0.02, Loss: 0.0499
Epoch 150, Treat Prop: 0.02, Loss: 0.0310
Epoch 150, Treat Prop: 0.02, Loss: 0.0645
Epoch 200, Treat Prop: 0.02, Loss: 0.0197
Epoch 200, Treat Prop: 0.02, Loss: 0.0581
Epoch 200, Treat Prop: 0.02, Loss: 0.0389
Epoch 200, Treat P

 39%|███▉      | 389/1000 [3:46:46<5:54:13, 34.79s/it]

0.026149839162826538
Seed: 389
Epoch 0, Treat Prop: 0.02, Loss: 0.3977
Epoch 0, Treat Prop: 0.02, Loss: 0.2494
Epoch 0, Treat Prop: 0.02, Loss: 0.3606
Epoch 0, Treat Prop: 0.02, Loss: 0.4547
Epoch 0, Treat Prop: 0.02, Loss: 1.2412
Epoch 50, Treat Prop: 0.02, Loss: 0.0540
Epoch 50, Treat Prop: 0.02, Loss: 0.1033
Epoch 50, Treat Prop: 0.02, Loss: 0.0687
Epoch 50, Treat Prop: 0.02, Loss: 0.0578
Epoch 50, Treat Prop: 0.02, Loss: 0.3395
Epoch 100, Treat Prop: 0.02, Loss: 0.0148
Epoch 100, Treat Prop: 0.02, Loss: 0.0327
Epoch 100, Treat Prop: 0.02, Loss: 0.0259
Epoch 100, Treat Prop: 0.02, Loss: 0.0328
Epoch 100, Treat Prop: 0.02, Loss: 0.0715
Epoch 150, Treat Prop: 0.02, Loss: 0.0357
Epoch 150, Treat Prop: 0.02, Loss: 0.0586
Epoch 150, Treat Prop: 0.02, Loss: 0.0168
Epoch 150, Treat Prop: 0.02, Loss: 0.0629
Epoch 150, Treat Prop: 0.02, Loss: 0.0588
Epoch 200, Treat Prop: 0.02, Loss: 0.0249
Epoch 200, Treat Prop: 0.02, Loss: 0.0206
Epoch 200, Treat Prop: 0.02, Loss: 0.0187
Epoch 200, Treat P

In [ ]:
GATE_hat_list.mean(axis=0)

In [ ]:
GATE_hat_list.std(axis=0)